***

## CSC424-202: Notes on Diffie-Hellman

author: burt rosenberg
<br>
date: 27 april 2020

***



### A bit of math

The mathematics involved is what happens with the integers mod n, when n is a prime.

We are interested in multiplication of integers mod n. Forget addition, and forget about the number 0.
Zero and any multiple of n is banished.

Of what remains, consider starting with some non-zero number mod n, call it g, and think about what happens
with the sequence,

$$
g, g^2, g^3, \ldots
$$

I will leave for the moment the reasoning behind this, in preference to set down the facts of the matter 
before looking for proofs, but obviously the numbers in this sequence will eventually exhaust the available
numbers, $1, 2, \ldots, n-1$, and repeat. And once a number repeats, what you have is the equation,

$$
g^i = g^k \pmod{n}
$$

hence the sequence will repeat from that point on,

$$
g^{i+1} = g\,g^i =  g\,g^k = g^{k+1} \pmod{n}
$$

and in particular, the marker for the repeat is $g^i= 1 \pmod{n}$.

__Experiment with this__

Take a non-zero $g$ and take its powers $g^2, g^3, \ldots$ until it
equals 1. That is your _orbit_, all the values $v$ for which there exists a $j$ such that $g^j = v \pmod{n}$.
What super-special is when any value of $v$ between 1 and $n-1$ has a $j$ such that $g^j = v \pmod{n}$.
In this case it is said that $g$ _generates_; and the $j$ is called the _discrete log_ of $v$.

Then write something that tries $g$ values to find one that generates.


In [1]:
import math 

def orbit(g,n,verbose=False):
    """
    the orbit of g in n, n a prime
    returns True if g is a generator, False otherwise
    """

    # this only makes sense for g and n relatively prime
    assert(math.gcd(g,n)==1)
    
    if verbose: print('\nmodulus: {}, generator: {}\norbit:'.format(n,g))
    a = g
    i = 1
    while a!=1:
        if verbose: print('{}: {}'.format(i,a))
        a = (a*g)%n
        i += 1
    if verbose: print('{}: {}\norbit done\n'.format(i,a))
    # if the orbit is the correct size, then g is a generator
    return i==(n-1)

def find_generator(n,verbose=False):
    """
    no magic, test 2, 3, 4 until finding a generator mod n
    """
    g = 2
    while not orbit(g,n):
        g += 1
    return g


In [2]:

modulus = 97

# if the generator is 1, little interesting happens, 
orbit(1,modulus,verbose=True)

# if the generator is -1, it becomes (-1)^2 = 1, even mod n, and the orbit ends.
orbit(-1,modulus,verbose=True)

# let's try 2, 3, and 4.

for g in range(2,7):
    orbit(g,modulus,verbose=True)
    
# note the orbit sizes discovered so far are 1, 2, 12, 24, 48 and 96.
# it is not an accident that all these numbers divide 96.
# but no time, sorry got to move on!

# what we want is a g such that the orbit is the biggest possible size, 96. 

generator = find_generator(modulus)



modulus: 97, generator: 1
orbit:
1: 1
orbit done


modulus: 97, generator: -1
orbit:
1: -1
2: 1
orbit done


modulus: 97, generator: 2
orbit:
1: 2
2: 4
3: 8
4: 16
5: 32
6: 64
7: 31
8: 62
9: 27
10: 54
11: 11
12: 22
13: 44
14: 88
15: 79
16: 61
17: 25
18: 50
19: 3
20: 6
21: 12
22: 24
23: 48
24: 96
25: 95
26: 93
27: 89
28: 81
29: 65
30: 33
31: 66
32: 35
33: 70
34: 43
35: 86
36: 75
37: 53
38: 9
39: 18
40: 36
41: 72
42: 47
43: 94
44: 91
45: 85
46: 73
47: 49
48: 1
orbit done


modulus: 97, generator: 3
orbit:
1: 3
2: 9
3: 27
4: 81
5: 49
6: 50
7: 53
8: 62
9: 89
10: 73
11: 25
12: 75
13: 31
14: 93
15: 85
16: 61
17: 86
18: 64
19: 95
20: 91
21: 79
22: 43
23: 32
24: 96
25: 94
26: 88
27: 70
28: 16
29: 48
30: 47
31: 44
32: 35
33: 8
34: 24
35: 72
36: 22
37: 66
38: 4
39: 12
40: 36
41: 11
42: 33
43: 2
44: 6
45: 18
46: 54
47: 65
48: 1
orbit done


modulus: 97, generator: 4
orbit:
1: 4
2: 16
3: 64
4: 62
5: 54
6: 22
7: 88
8: 61
9: 50
10: 6
11: 24
12: 96
13: 93
14: 81
15: 33
16: 35
17: 43
18: 75
19: 9
20: 

In [3]:
import random


class Alice:
    
    def __init__(self,modulus,generator):
        self.modulus = modulus
        self.generator = generator
        self.secret = 0
        self.shared = 0

    def protocol_step_one(self):
        self.secret = random.randint(2,self.modulus-1)
        return (self.generator**self.secret%self.modulus)
        
    def protocol_step_two(self,bobs_message):
        self.shared = (bobs_message**self.secret)%self.modulus

    def show_all(self):
        print('\nAlice:\n\tmodulus: {}\n\tgenerator: {}\n\tsecret: {}\n\tshared: {}'.format(
            self.modulus, self.generator, self.secret, self.shared ))

        
class Bob:
    
    def __init__(self,modulus,generator):
        self.modulus = modulus
        self.generator = generator
        self.secret = 0

    def protocol_step_one(self):
        self.secret = random.randint(2,self.modulus-1)
        return (self.generator**self.secret%self.modulus)
        
    def protocol_step_two(self,alices_message):
        self.shared = (alices_message**self.secret)%self.modulus

    def show_all(self):
        print('\nBob:\n\tmodulus: {}\n\tgenerator: {}\n\tsecret: {}\n\tshared: {}'.format(
            self.modulus, self.generator, self.secret, self.shared ))


def diffie_hellman_protocol(modulus,generator):
    alice = Alice(modulus,generator)
    bob = Bob(modulus,generator)
    alice_says = alice.protocol_step_one()
    bob_says = bob.protocol_step_one()
    alice.protocol_step_two(bob_says)
    bob.protocol_step_two(alice_says)

    return(alice,bob)



### Try out the protocol


In [4]:
alice,bob = diffie_hellman_protocol(modulus,generator)
alice.show_all()
bob.show_all()


Alice:
	modulus: 97
	generator: 5
	secret: 17
	shared: 19

Bob:
	modulus: 97
	generator: 5
	secret: 33
	shared: 19
